# Automated Globe Prediction in CT images of the Orbits
## Axial CT images of the orbits 

In [ ]:
"""
UNIVERSITY OF ARIZONA
Author: Lavanya Umapathy
Date: 
Description: Script to evaluate/test using a saved CNN model. 
The MRes-UNET2D model uses Axial CT images of the orbits to predict globe masks and quantify globe volumes.
"""

In [ ]:
from matplotlib import pyplot as plt
import time, sys
from skimage import measure
import Utilities as utils

In [ ]:
# path to the saved pre-trained model
model_loadPath = '../PretrainedModel/MRes_UNET2D.h5'

# path to a dicom directory containing CT images. Replace this with actual data directory
dcm_loadPath = '../Data/SubjectFolderName/'

output_shape = (512,512)
WL = 50  # in hounsfield units, for window level
WW = 200  # in hounsfield units, for window width
dicom_srchstr = 'IM*'
gpu_number = '0'

In [ ]:
# load the CT DICOM series into img (Height x Width x Number of Slices)
img = utils.loadDicomSeries_sorted(dcmPath,dicom_srchstr)

# Get pixel size for volume calculations
pixdim = utils.getPixDims_Dicom(dcmPath,dicom_srchstr)

# Pre-process CT images of the orbits
img  = utils.preProcess_orbitalCT(img,output_shape,WL=WL,WW=WW)

In [ ]:
# Load the pretrained MRes-UNET2D model
model = utils.loadSavedModel(modelLoadPath,gpu_number)

# Predict masks for globes using MRes-UNET2D model
start_pred = time.time()
predictedGlobes = utils.predictGlobes(model, img)
end_pred = time.time()
print("Time Elapsed prediction in seconds: ",round(end_pred - start_pred,4))

In [ ]:
# Display the predicted Globe contours
img_idx = 2    # select an image to display the contours on
contours_pred = measure.find_contours(predImg[:,:,idx], 0.25)
fig, ax = plt.subplots()
ax.imshow(img[:,:,idx], interpolation='nearest', cmap=plt.cm.gray)
for n, contour in enumerate(contours_pred):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=1.5, color='blue')

In [ ]:
# Print evaluation stats
computeGlobeStats(predictedGlobes,pixdim)